In [ ]:

import os
import sys
import pickle
import warnings
import time
import logging
import glob


import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from iminuit import Minuit
from matplotlib import ticker
from matplotlib.legend import Legend


from utils import *
from AnalyseDefects import AnalyseDefects

plt.style.use('sg_article')

In [ ]:
def gen_analysis_dict(LL, mode):

    dshort = dict(path = f'C:\\Users\\Simon Andersen\\Documents\\Uni\\Speciale\\Hyperuniformity\\nematic_analysis{LL}_LL0.05', \
              suffix = "short", priority = -1, LX = LL, Nframes = 181)
    dlong = dict(path = f'C:\\Users\\Simon Andersen\\Documents\\Uni\\Speciale\\Hyperuniformity\\nematic_analysis{LL}_LL0.05_long', \
                suffix = "long", priority = 1, LX = LL, Nframes = 400)
    priority_vl = 2 if LL == 512 else 3
    dvery_long = dict(path = f'C:\\Users\\Simon Andersen\\Documents\\Uni\\Speciale\\Hyperuniformity\\nematic_analysis{LL}_LL0.05_very_long',\
                    suffix = "very_long", priority = priority_vl, LX = LL, Nframes = 1500)
    dvery_long2 = dict(path = f'C:\\Users\\Simon Andersen\\Documents\\Uni\\Speciale\\Hyperuniformity\\nematic_analysis{LL}_LL0.05_very_long_v2',\
                    suffix = "very_long2", priority = 3 if priority_vl == 2 else 2, LX = LL, Nframes = 1500)

    if mode == 'all':
        if LL == 2048:
            defect_list = [dshort, dlong]
        else:
            defect_list = [dshort, dlong, dvery_long, dvery_long2] if LL in [256, 512] else [dshort, dlong, dvery_long]
    else:
        defect_list = [dshort]
    
    return defect_list

LL = 2048
output_path = f'data\\nematic_analysis{LL}_LL0.05'
mode = 'all' # 'all' or 'short'

defect_list = gen_analysis_dict(LL, mode)

ad = AnalyseDefects(defect_list, output_path=output_path)

extract = False
analyze = False
merge = False

In [ ]:
def calc_autocorr(x, axis = 0, conv_list = None):

    conv_list = x.shape[1] * [0] if conv_list is None else conv_list

    x_av = np.nan * np.zeros(x.shape[1])
    auto_corr = np.nan * np.zeros_like(x)

    for i in range(x.shape[1]):
        x_av[i] = np.nanmean(x[conv_list[i]:,i])
        X = x[conv_list[i]:,i] - x_av[i]
        auto_corr[:,i] = np.correlate(X, X, mode='full') / np.sum(X[:]**2)

    X = (x - x_av) 

  # result = np.correlate(x - av, x - av, mode='full') / var ## np.nansum((x)**2)
    result = np.correlate(X, X, mode='full') / np.sum(X[:]**2)
    return result[result.size // 2:] #/ normalization

def calc_autocorr(x,conv_list = None):

    conv_list = x.shape[1] * [0] if conv_list is None else conv_list

    x_av = np.nan * np.zeros(x.shape[1])
    auto_corr = np.nan * np.zeros_like(x)

    for i in range(x.shape[1]):
        x_av[i] = np.nanmean(x[conv_list[i]:,i])
        X = x[conv_list[i]:,i] - x_av[i]
        X_corr = np.correlate(X, X, mode='full') / np.sum(X[:]**2)
        X_corr = X_corr[X_corr.size // 2:]
        auto_corr[-X_corr.size:,i] = X_corr
    return auto_corr#[auto_corr.shape[0] // 2:] #/ normalization


In [ ]:
Ndataset_list = [0,-1] # [0,1, 2]
act_idx_bounds_list = [[6,None], [0,None]] #] [[11,-3], [-2,None], [0,None]]
fig, ax = plt.subplots(ncols=2, figsize=(16,12))
ax = ax.flatten()
for k, Ndataset in enumerate(Ndataset_list):
    defect_arr = ad.get_arrays_full(Ndataset=Ndataset)[0]
    conv_list = ad.conv_list[Ndataset]
  #  conv_list = defect_arr.shape[1] * [0]
    act_list = ad.act_list[Ndataset]
    auto_corr = np.zeros_like(defect_arr)
    Ndist = np.zeros(defect_arr.shape[1])
    lag = 10
    for i in range(defect_arr.shape[-1]):
        auto_corr[:, :, i] = calc_autocorr(defect_arr[:,:,i], conv_list)#np.apply_along_axis(calc_autocorr, axis = 0, arr = defect_arr[:, :, i])

   # fig, ax = plt.subplots(ncols=2, figsize=(16,12))
   # ax = ax.flatten()
    auto_corr_mean = np.nanmean(auto_corr, axis = -1)
    auto_corr_std = np.nanstd(auto_corr, axis=-1) / np.sqrt(defect_arr.shape[-1])
    Ndist 
    i0 = 0
    for i, act in enumerate(act_list[act_idx_bounds_list[k][0]:act_idx_bounds_list[k][1]]):
        i += i0
        ax[0].errorbar(act, auto_corr_mean[conv_list[i] + lag,i], auto_corr_std[conv_list[i] + lag,i], fmt='o', label=f'act = {act}')
        ax[1].errorbar(range(defect_arr.shape[0]),auto_corr_mean[:,i], auto_corr_std[:,i], label=f'act = {act}',alpha=.3)
        Ndist[i] = (1+2*(auto_corr_mean[conv_list[i] + 1:,i]**2).sum(axis=0))
    ax[1].legend(ncol=2,fontsize=12)
    ax[0].legend(ncol=2,fontsize=12)
    #Ndist = (1+2*(auto_corr_mean[1:,:]**2).sum(axis=0))
    Ndist, defect_arr.shape[0] * defect_arr.shape[-1] / Ndist
ax[0].set_xscale('log') 
ax[1].set_xscale('log')

In [ ]:
Ndataset=-1
defect_arr = ad.get_arrays_full(Ndataset=Ndataset)[0]
conv_list = ad.conv_list[Ndataset]
conv_list = defect_arr.shape[1] * [0]
act_list = ad.act_list[Ndataset]
auto_corr = np.zeros_like(defect_arr)
Ndist = np.zeros(defect_arr.shape[1])
lag = 1
for i in range(defect_arr.shape[-1]):
    auto_corr[:, :, i] = calc_autocorr(defect_arr[:,:,i], conv_list)#np.apply_along_axis(calc_autocorr, axis = 0, arr = defect_arr[:, :, i])

fig, ax = plt.subplots(ncols=2, figsize=(16,12))
ax = ax.flatten()
auto_corr_mean = np.nanmean(auto_corr, axis = -1)
auto_corr_std = np.nanstd(auto_corr, axis=-1) / np.sqrt(defect_arr.shape[-1])
Ndist 
i0 = 0
for i, act in enumerate(act_list[i0:]):
    i += i0
    ax[0].errorbar(act, auto_corr_mean[conv_list[i] + lag,i], auto_corr_std[conv_list[i] + lag,i], fmt='o', label=f'act = {act}')
    ax[1].errorbar(range(defect_arr.shape[0]),auto_corr_mean[:,i], auto_corr_std[:,i], label=f'act = {act}',alpha=.3)
    Ndist[i] = (1+2*(auto_corr_mean[conv_list[i] + 1:,i]**2).sum(axis=0))
ax[1].legend(ncol=2,fontsize=12)
ax[0].legend(ncol=2,fontsize=12)
#Ndist = (1+2*(auto_corr_mean[1:,:]**2).sum(axis=0))
Ndist, defect_arr.shape[0] * defect_arr.shape[-1] / Ndist

In [ ]:
Ndataset=-1
defect_arr = ad.get_arrays_full(Ndataset=Ndataset)[0]

act_list = ad.act_list[Ndataset]
auto_corr = np.zeros_like(defect_arr)
lag = 60
for i in range(defect_arr.shape[-1]):
    auto_corr[:, :, i] = np.apply_along_axis(calc_autocorr, axis = 0, arr = defect_arr[:, :, i])

fig, ax = plt.subplots(ncols=2, figsize=(16,12))
ax = ax.flatten()
auto_corr_mean = np.nanmean(auto_corr, axis = -1)
auto_corr_std = np.nanstd(auto_corr, axis=-1) / np.sqrt(defect_arr.shape[-1])
ax[0].errorbar(act_list, auto_corr_mean[lag], auto_corr_std[lag], fmt='.-')
for i, act in enumerate(act_list):
    ax[1].errorbar(range(defect_arr.shape[0]),auto_corr_mean[:,i], auto_corr_std[:,i], label=f'act = {act}',alpha=.3)
ax[1].legend(ncol=2,fontsize=12)
ax[0].legend(ncol=2,fontsize=12)
Ndist = (1+2*(auto_corr_mean[1:,:]**2).sum(axis=0))
Ndist, defect_arr.shape[0] * defect_arr.shape[-1] / Ndist